In [ ]:
import math
import numpy as np
from numpy import pi
from numpy import random
from numpy import linalg
import csv

this is used to initialize the state parameters 

In [ ]:
# initial weight parameters
mu = np.zeros(6)
sigma = 2
theta_initial = np.array([0.0,0.0,0.0,0.0,0.0,0.0,2.0])#np.zeros(7) #0~5 mu; 6:sigma

# initial state parameter
omega = 0
phi = 0
d = 0
kappa1 = 0
kappa2 = 0
l = 0
s_initial = np.array([0,0.19455,0,0.01006,0.01006,1])#np.zeros(6) #0:omege; 1:phi; 2:d; 3:kappa1; 4:kappa2; 5:l

# init parameter for E_location
tau1 = 0.5
tau2 = 0.5
zeta1 = 1/3
zeta2 = 1/3
zeta3 = 1/3
discount = 0.99

# init for kappa
alpha = 0.05
curv1 = 10
curv2 = 10
curv = [curv1, curv2]

# init geometry
width = 2
R_brush = width/2
R_med = 10
R_inner = R_med - width/2
R_outer = R_med + width/2

tau1=tau2=0.5 for upright brush style; note the center of the c should not exceed the inner and outer boundary

In [ ]:
def E_location(STT):
  if STT[2] >= -0.2 and STT[2] <= 0.2:
    R = tau1 * np.absolute(STT[0]) + tau2 * np.absolute(STT[2]) 
  else:
    R = tau1 * np.absolute(STT[0]) + tau2 * (np.absolute(STT[2]) + 5)
  return R


In [ ]:
def sq_change(Xt,Xt_minus):
    if Xt==Xt_minus and Xt==0:
        R = 1
    else:
        R = (Xt-Xt_minus)**2/(np.absolute(Xt)+np.absolute(Xt_minus))**2
        
    return R

In [ ]:
def E_posture(STT,STT_prev):
    # need STT and STT_previous
    R = zeta1*sq_change(STT[0],STT_prev[0]) + zeta2*sq_change(STT[1],STT_prev[1]) + zeta3*sq_change(STT[2],STT_prev[2])

    return R     

In [ ]:
def calc_kappa(curv):
  kappa1 = (2 / pi) * np.arctan(alpha/(np.sqrt(curv1)))
  kappa2 = (2 / pi) * np.arctan(alpha/(np.sqrt(curv2)))

  return kappa1, kappa2

In [ ]:
# modified
# def step_c(xt, yt, act, STT):
#   # tangent direction of the arc or the velocity vector of the brush stroke
#   vx = yt
#   vy = -xt
#   tangent = np.array([vx, vy])
#   #normalize the velocity
#   tangent = tangent/np.linalg.norm(tangent)
#   #rotation according to action
#   w_t = STT[0]
#   act_rotation = np.array([[np.cos(w_t+act), -np.sin(w_t+act)],
#                            [np.sin(w_t+act),  np.cos(w_t+act)]])
#   velocity = np.dot(act_rotation, tangent)
#   step_center = R_brush/3 * velocity
#   x_next = xt + step_center[0]
#   y_next = yt + step_center[1]
#   # Calculate the new w
#   tangent_next = np.array([y_next, -x_next])
#   tangent_next /= np.linalg.norm(tangent_next)
#   w_next = np.arccos(np.dot(velocity,tangent_next))
  
#   # update d, R_brush is fixed
#   dist_offset = np.sqrt(x_next**2 + y_next**2) - R_med
#   d_next = dist_offset/R_brush
 
#   return x_next, y_next, w_next, d_next

In [ ]:
# Query boundary, the shape is a half circle, covering the 1st,4th quadrant
# need R_brush is fixed as the 1/2 width of this shape

def step_c(xt, yt, act, STT):
  # tangent direction of the arc or the velocity vector of the brush stroke
  vx = yt
  vy = -xt
  tangent = np.array([vx, vy])
  #normalize the velocity
  tangent = tangent/np.linalg.norm(tangent)
  #rotation according to action
  w_t = STT[0]
  act_rotation = np.array([[np.cos(w_t+act), -np.sin(w_t+act)],
                           [np.sin(w_t+act),  np.cos(w_t+act)]])
  velocity = np.dot(act_rotation, tangent)
  step_center = R_brush/3 * velocity
  x_next = xt + step_center[0]
  y_next = yt + step_center[1]
  # Calculate the new w
  tangent_next = np.array([y_next, -x_next])
  tangent_next /= np.linalg.norm(tangent_next)

  cosTh = np.dot(tangent_next, velocity)

  sinTh = np.cross(tangent_next, velocity)
  w_next = np.arctan2(sinTh, cosTh)
  
  # update d, R_brush is fixed
  dist_offset = np.sqrt(x_next**2 + y_next**2) - R_med
  d_next = dist_offset/R_brush
 
  return x_next, y_next, w_next, d_next

In [ ]:
def query_boundary(Cx, Cy):
  ft_head = np.sqrt(Cx**2 + Cy**2)
  if ft_head >= R_outer or ft_head <= R_inner:
    return True
  else:
    return False


In [ ]:
def action_policy(Theta, STT, Cx, Cy):
  max_attempt_num = 5
  attempt = 1
  action = random.normal(np.dot(Theta[0:6], STT),Theta[6])
  xtt, ytt, _, _ = step_c(Cx, Cy, action, STT)
  while query_boundary(xtt, ytt):
    action = random.normal(np.dot(Theta[0:6], STT),Theta[6])
    xtt, ytt, _, _ = step_c(Cx, Cy, action, STT)
    attempt = attempt + 1
    if attempt > max_attempt_num:
      break
      
  return action
    

In [ ]:
def Imm_reward(st_prev, st, l_next):
    if l_next == 0:
        return 0
    else:
        return ((np.absolute(st[3])+np.absolute(st[4])+2)/(E_location(st)+E_posture(st,st_prev))) # E_pos need st_prev

In [ ]:
def gradient_mu(act, STT, Theta):
  nabla_mu = ((act - np.dot(Theta[0:6], STT))/ Theta[6]**2) * STT
  return nabla_mu

In [ ]:
def gradient_sigma(act, STT, Theta):
    nabla_sig = ((act - np.dot(Theta[0:6],STT))**2 -Theta[6]**2)/( Theta[6]**3)
    return nabla_sig


In [ ]:
def update_Q(Cx,Cy,Cx_prev,Cy_prev,cq):
  extend = 0
  if Cy == 0:
    Qx = (R_med **2 - cq **2 + Cx **2) / (2 * Cx)
    if R_med **2 < Qx **2:
      Qx = 10
      Qy = 0
      cq = np.absolute (Qx - Cx)
      extend = 1
    else:
       Qy = np.sqrt(R_med **2 - Qx **2)
    return Qx, Qy, extend, cq

  res = (R_med **2 - cq **2 + Cx**2 +Cy ** 2)/(2 * Cy)
  a = 1 + (Cx/Cy) ** 2
  b = -2 * res * Cx/Cy
  c = res**2 - R_med **2
  
  delta = b**2 - 4 * a * c
  
  if delta < 0 :
    extend = 1
    if Cx == 0:
      Qx = 0 
      Qy = 10
      cq = np.absolute(Qy - Cy)
    else:
      Qx = R_med ** 2/ (1 + (Cy/Cx) ** 2)
      Qy = Cy * Qx / Cx
      cq = np.sqrt ((Qx - Cx) ** 2 +(Qy - Cy) ** 2)
    return Qx,Qy,extend,cq

  Qx_1 = (-b + np.sqrt(delta))/(2 * a)
  Qy_1 = res  - Cx/Cy * Qx_1
    
  Qx_2 = (-b - np.sqrt(delta))/(2 * a)
  Qy_2 = res  - Cx/Cy * Qx_2
    
  distance1 = (Qx_1 - Cx_prev) ** 2 + (Qy_1 - Cy_prev) ** 2
  distance2 = (Qx_2 - Cx_prev) ** 2 + (Qy_2 - Cy_prev) ** 2
    
  if distance1 < distance2:
    return Qx_1,Qy_1,extend,cq
  else:
    return Qx_2,Qy_2,extend,cq
    

Solution to get update Q
https://drive.google.com/file/d/13VFuMbSeE8zjzflMqvUPkxevm3YaVPmT/view?usp=sharing

In [ ]:
def find_next_phi(Cx, Cy, Qx, Qy):
  CQ_vector = np.array([(Qx- Cx),(Qy - Cy)])
  #normalize CQ
  CQ_vector = CQ_vector / np.linalg.norm(CQ_vector)

  x = -Cy
  y = Cx
  tangent = np.array([x, y])
  #normalize the velocity
  tangent = tangent/np.linalg.norm(tangent)

  sin_phi = np.cross(tangent, CQ_vector)
  cos_phi = np.dot(tangent, CQ_vector)

  phi = np.arctan2(sin_phi, cos_phi)

  return phi

In [ ]:
def find_l(Cx,Cy,Cx_prev,Cy_prev):
  if Cx < Cx_prev:
    l = 0
  elif Cx < 0 or Cy <0:
    l = 0
  else:
    l = 1
  
  return l



In [ ]:
# def find_l(Cx,Cy,Cx_prev,Cy_prev):
#   if Cy == 0 :
#     if Cx <= 0:
#       angle = -np.pi/2
#     else:
#       angle = np.pi/2
#   else:
#     angle = np.arctan(Cx/Cy)
    
#   if Cy_prev == 0 :
#     if Cx_prev <= 0:
#       angle_prev = -np.pi/2
#     else:
#       angle_prev = np.pi/2
#   else:
#     angle_prev = np.arctan(Cx/Cy) 
        
        
       
#   if angle > angle_prev and angle <= np.pi/2:
#     return 1
    
#   else:
#     return 0

called at t = T, to determine whether next episode should start from current location or start from somewhere near the S point; depending on whether point C is within the boundary or if it is close to the goal position

In [ ]:
def agent_restart(CQ_len, Cx, Cy, Qx, Qy):
  dist_to_end = np.sqrt((10 - Cx)**2 + Cy**2)
  threshold = 4
  flag = False
  if (CQ_len > threshold) or (dist_to_end < 1) or query_boundary(Cx, Cy):
    Cx = 3.787
    # Cx = -10 + 0.1 * random.rand(1)
    Cy = 9.225
    # Cy = 0 + 0.1 * random.rand(1)
    Qx = 0
    Qy = 10
    # return Cx.squeeze(), Cy.squeeze()
    flag = True
    return Cx, Cy, Qx, Qy, flag

  else:
    return Cx, Cy, Qx, Qy, flag


In [ ]:
def main():
  trials = 80
  episodes = 300
  T = 20
  
  Reward = np.zeros([trials,episodes]) #total reward over trajectory
  num_candidate = 4
  action_candidate = np.zeros(num_candidate)
  candidate_reward = np.zeros(num_candidate)
  Action = 0

  nabla_mu_J = 0
  nabla_sigma_J = 0

  Theta = theta_initial



  for i in range(trials):
    state_prev = s_initial
    state_curr = s_initial
    b_star = 0
    b_star_numerator = 0
    b_star_denominator = 0
    xt = 3.787
    yt = 9.225
    qx = 0
    qy = 10
    sum_sigma_T = np.zeros(episodes)
    sum_mu_T = np.zeros([episodes, 6])
    CQ = R_brush / np.sin(15/180 * pi)

    for e in range(episodes):
      for t in range(T):
        for c in range(num_candidate):
          action_candidate[c]  = action_policy(Theta, state_curr, xt, yt)
          xtt, ytt, _, _ = step_c(xt, yt, action_candidate[c], state_curr)
          L = find_l(xtt, ytt, xt, yt)
          candidate_reward[c] = Imm_reward(state_prev, state_curr, L)
        Action = action_candidate[np.argmax(candidate_reward)]
        Reward[i,e] = Reward[i,e] + (discount ** t) * max(candidate_reward)
        xt_c = xt
        yt_c = yt
        xt, yt, w_n, d_n = step_c(xt, yt, Action, state_curr)
        qx, qy, Extend, CQ = update_Q(xt, yt, xt_c, yt_c,CQ)
        # n_ext = n_ext + Extend
        PHI = find_next_phi(xt,yt,qx, qy)
        L = find_l(xt, yt, xt_c, yt_c)
        K1 , K2 = calc_kappa(curv)
        state_prev = state_curr
        state_curr = np.array([w_n, PHI, d_n, K1, K2, L])
        sum_mu_T[e] = sum_mu_T[e] + gradient_mu(Action, state_curr, Theta)
        sum_sigma_T[e] = sum_sigma_T[e] + gradient_sigma(Action,state_curr,Theta)
      xt, yt, qx, qy, f = agent_restart(CQ, xt, yt, qx, qy)
      if f:
          CQ = R_brush / np.sin(15/180 * pi)
          state_prev = s_initial
          state_curr = s_initial

      sub_b = np.append(sum_mu_T[e], sum_sigma_T[e])
      norm_term = linalg.norm(sub_b)**2
      b_star_numerator += Reward[i,e] * norm_term
      b_star_denominator += norm_term
      
    b_star =  b_star_numerator/ b_star_denominator

    nabla_mu_J = np.dot(Reward[i] - b_star, sum_mu_T) / episodes
    nabla_sigma_J = np.dot(Reward[i] - b_star, sum_sigma_T) /episodes
    g_theta = np.append(nabla_mu_J,nabla_sigma_J)
    Theta[0:6] += (0.05/linalg.norm(g_theta)) * nabla_mu_J
    Theta[6] += (0.05/linalg.norm(g_theta)) * nabla_sigma_J



  with open('submission1.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',', quotechar='\"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['iteration', 'Reward'])
    for i in range(trials):
      writer.writerow([i, np.average(Reward[i])])




    

In [ ]:
main()

if c goes to far, after one episode ends, go back to original point